In [1]:
import pandas as pd
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
import os

# Current work
print(os.getcwd())
# Change directory 2 level up
os.chdir("..")
os.chdir("..")
os.chdir("..")
print(os.getcwd())
# Look for the file in the directory
print(os.listdir())

/tmp/ipykernel_24551/4120722875.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


/home/jose/Git/SE-AP/experiments/experiment_2/notebooks
/home/jose/Git/SE-AP
['models', 'todo.md', '.pre-commit-config.yaml', 'demos', '.gitignore', 'experiments', 'include', '.git', '.env', 'README.md', '.env-example', 'LICENSE', 'model_technique_pathology_scores.csv']


In [2]:
model = "llama"
prompt_technique = "chain"
temperature = 0.7

In [3]:
data = pd.read_csv(f'experiments/experiment_2/results_with_distances/{model}/results_with_distances_{prompt_technique}_{model}_{temperature}_distances.csv')
data[data['Iteration'] == 1]

,Induced Pathology,Category,Surveyed Word,Related 1,Related 2,Related 3,Related 4,Related 5,Related 6,Related 7,...,Sequential Distance 4,Sequential Distance 5,Sequential Distance 6,Sequential Distance 7,Sequential Distance 8,Sequential Distance 9,Sequential Distance 10,Average Sequential Distance,STD Sequential Distance,Simplex Volume
0,No Pathology,Concrete,house,home,family,living space,comfort,security,memories,decor,...,0.284060,0.328870,0.289298,0.416784,0.502047,0.369640,0.202819,0.410342,0.166186,21.287108
1,No Pathology,Concrete,car,vehicle,transportation,driving,freedom,independence,convenience,speed,...,0.188385,0.129086,0.313376,0.411343,0.277774,0.252515,0.331893,0.375261,0.202436,7.240148
2,No Pathology,Concrete,coat,jacket,sweater,winter,warmth,style,fashion,protection,...,0.364448,0.330856,0.347206,0.295246,0.388301,0.364418,0.375295,0.414714,0.122846,14.560554
3,No Pathology,Concrete,necklace,jewelry,adornment,accessory,beauty,elegance,shine,sparkle,...,0.344022,0.338523,0.285243,0.498664,0.596635,0.333023,-0.022674,0.407450,0.214722,9.701641
4,No Pathology,Concrete,sofa,couch,seating,comfort,relaxation,living room,furniture,design,...,0.326478,NaN,0.669709,0.275740,0.321561,0.408812,0.638413,0.487302,0.187368,18.396814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,Social Anxiety,Abstract,insecurity,fear,anxiety,nervousness,apprehension,unease,vulnerability,susceptibility,...,0.490726,0.618700,0.585519,0.280753,0.518855,0.612597,0.489297,0.530999,0.100543,3.670347
1596,Social Anxiety,Abstract,indifference,apathy,disinterest,lack of enthusiasm,nonchalance,disregard,unconcern,dispassion,...,0.630588,0.708560,0.624851,0.422060,NaN,0.428169,0.343319,0.592322,0.174663,11.172335
1597,Social Anxiety,Abstract,infidelity,betrayal,deceit,disloyalty,cheating,unfaithfulness,faithlessness,disloyalty,...,0.622774,0.698059,0.516115,0.500013,0.568996,0.437634,0.433398,0.547139,0.087147,4.272550
1598,Social Anxiety,Abstract,aggressiveness,hostility,anger,violence,belligerence,malice,resentment,aggression,...,0.604582,0.391540,0.442538,0.709105,0.604582,0.578831,0.413181,0.511950,0.120583,7.241034


In [4]:
distances_columns = [col for col in data.columns if 'Sequential Distance' in col]
numeric_data = data[['Induced Pathology', 'Surveyed Word'] + distances_columns]
grouped_data = numeric_data.groupby(['Induced Pathology', 'Surveyed Word']).mean().reset_index()

pca_results = []
induced_pathologies = grouped_data['Induced Pathology'].unique()
variance_by_pathology_iteration = {}
n_components = 10

for pathology in induced_pathologies:
    variance_by_pathology_iteration[pathology] = []
    for iteration in data['Iteration'].unique():
        iteration_data = data[(data['Induced Pathology'] == pathology) & (data['Iteration'] == iteration)]
        sequential_distances_data = iteration_data.filter(like='Sequential Distance').dropna()
        if len(sequential_distances_data) >= n_components:
            pca = PCA(n_components=n_components)
            pca.fit(sequential_distances_data)
            varianza_acumulada = np.cumsum(pca.explained_variance_ratio_) * 100
            variance_by_pathology_iteration[pathology].append(varianza_acumulada)

for pathology in variance_by_pathology_iteration:
    variance_by_pathology_iteration[pathology] = np.array(variance_by_pathology_iteration[pathology])

marcadores = {'Depression': 'o', 'Trait Anxiety': 's', 'Eating Disorder': '^', 'Alcohol Addiction': 'D',
              'Impulsivity': 'v', 'Schizophrenia': '<', 'Obsessive Compulsive Disorder': '>', 'Apathy': 'P',
              'Social Anxiety': '*', 'No Pathology': 'X'}

fig, ax = plt.subplots(figsize=(14, 8))
colores = plt.cm.tab10(np.linspace(0, 1, len(induced_pathologies)))

for i, pathology in enumerate(variance_by_pathology_iteration):
    mean_variance = np.mean(variance_by_pathology_iteration[pathology], axis=0)
    std_variance = np.std(variance_by_pathology_iteration[pathology], axis=0)
    components = np.arange(1, n_components + 1)
    ax.plot(components, mean_variance, label=pathology, color=colores[i], marker=marcadores[pathology])
    ax.fill_between(components, mean_variance - std_variance, mean_variance + std_variance, color=colores[i], alpha=0.1)

ax.axhline(y=80, color='r', linestyle='--', label='80% Variance Explained')

ax.set_xlabel('Dimensions', fontsize=22)
ax.set_ylabel('Variance explained', fontsize=22)
ax.set_title('Semantic Dimensionality', fontsize=25)
ax.legend(loc='best', fontsize=18)

ax.set_xticks(np.arange(1, 11))

ax.tick_params(axis='both', which='major', labelsize=18)

plt.tight_layout()
plt.savefig(f'experiments/experiment_2/figs/{model}/{temperature}/exp2_semantic_dimensionality_{prompt_technique}_{model}_{temperature}.png')
plt.show()